In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.utils import algorithm_globals
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.neural_networks import EstimatorQNN, SamplerQNN
from sklearn.model_selection import train_test_split
from qiskit_algorithms.optimizers import COBYLA
import tensorflow as tf
import collections

# Define the convolutional circuit
def conv_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)
    target.cx(1, 0)
    target.rz(np.pi / 2, 0)
    return target

# Define the convolutional layer
def conv_layer(num_qubits, param_prefix):
    qc = QuantumCircuit(num_qubits, name="Convolutional Layer")
    qubits = list(range(num_qubits))
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits * 3)
    for q1, q2 in zip(qubits[0::2], qubits[1::2]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3
    for q1, q2 in zip(qubits[1::2], qubits[2::2] + [0]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, qubits)
    return qc

# Define the pooling circuit
def pool_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)

    return target

# Define the pooling layer
def pool_layer(sources, sinks, param_prefix):
    num_qubits = len(sources) + len(sinks)
    qc = QuantumCircuit(num_qubits, name="Pooling Layer")
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits // 2 * 3)
    for source, sink in zip(sources, sinks):
        qc = qc.compose(pool_circuit(params[param_index : (param_index + 3)]), [source, sink])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, range(num_qubits))
    return qc

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

# Filter for digits 0, 1, 2, and 3
def filter_0123(x, y):
    keep = (y == 0) | (y == 1) | (y == 2) | (y == 3)
    x, y = x[keep], y[keep]
    return x, y
x_train, y_train = filter_0123(x_train, y_train)
x_test, y_test = filter_0123(x_test, y_test)

# Resize images to 4x4
x_train_small = tf.image.resize(x_train, (4,4)).numpy()
x_test_small = tf.image.resize(x_test, (4,4)).numpy()

# Remove images with contradicting labels
def remove_contradicting(xs, ys):
    mapping = collections.defaultdict(set)
    orig_x = {}
    for x, y in zip(xs, ys):
       orig_x[tuple(x.flatten())] = x
       mapping[tuple(x.flatten())].add(y)
    
    new_x = []
    new_y = []
    for flatten_x in mapping:
      x = orig_x[flatten_x]
      labels = mapping[flatten_x]
      if len(labels) == 1:
          new_x.append(x)
          new_y.append(next(iter(labels)))
      else:
          pass
    
    return np.asarray(new_x), np.asarray(new_y)

resized_train_images, train_labels = remove_contradicting(x_train_small, y_train)
resized_test_images, test_labels = remove_contradicting(x_test_small, y_test)

# Define the feature map
feature_map = ZFeatureMap(16)

# Define the quantum circuit (same structure as before)
ansatz = QuantumCircuit(16, name="Ansatz")
ansatz.compose(conv_layer(16, "c1"), list(range(16)), inplace=True)
ansatz.compose(pool_layer([0, 1, 2, 3, 4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14, 15], "p1"), list(range(16)), inplace=True)
ansatz.compose(conv_layer(8, "c2"), list(range(8, 16)), inplace=True)
ansatz.compose(pool_layer([0, 1, 2, 3], [4, 5, 6, 7], "p2"), list(range(8, 16)), inplace=True)
ansatz.compose(conv_layer(4, "c3"), list(range(12, 16)), inplace=True)
ansatz.compose(pool_layer([0, 1], [2, 3], "p3"), list(range(12, 16)), inplace=True)
ansatz.compose(conv_layer(2, "c4"), list(range(14, 16)), inplace=True)
ansatz.compose(pool_layer([0], [1], "p4"), list(range(14, 16)), inplace=True)

# Combine feature map and ansatz
circuit = QuantumCircuit(16)
circuit.compose(feature_map, range(16), inplace=True)
circuit.compose(ansatz, range(16), inplace=True)

# Define the QNN
# Use Aer qasm_simulator and specify shots
quantum_instance = QuantumInstance(Aer.get_backend('qasm_simulator'), shots=1024, seed_simulator=42)
sampler = Sampler()
qnn = SamplerQNN(
    circuit=circuit.decompose(),
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    sampler=sampler,
)

# Define the callback function
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

# Define the classifier
classifier = NeuralNetworkClassifier(
    qnn,
    optimizer=COBYLA(maxiter=5),  # Consider increasing iterations
    callback=callback_graph,
)

# Prepare the training data (one-hot encoding for 4 classes)
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
train_labels_onehot = onehot_encoder.fit_transform(train_labels.reshape(-1, 1))
x_reshaped = resized_train_images.reshape(-1, 16)

# Reduce training data size for faster testing
x_reshaped = x_reshaped[:50] 
train_labels_onehot = train_labels_onehot[:50]

# Train the classifier
objective_func_vals = []
classifier.fit(x_reshaped, train_labels_onehot)
print(f"Accuracy from the train data : {np.round(100 * classifier.score(x_reshaped, train_labels_onehot), 2)}%")

# Prepare the test data (one-hot encoding)
test_labels_onehot = onehot_encoder.fit_transform(test_labels.reshape(-1, 1))
x1_reshaped = resized_test_images.reshape(-1, 16)
x1_reshaped = x1_reshaped[:50] 
test_labels_onehot = test_labels_onehot[:50]

# Evaluate the classifier
y_predict = classifier.predict(x1_reshaped)
print(f"Accuracy from the test data : {np.round(100 * classifier.score(x1_reshaped, test_labels_onehot), 2)}%")

# Visualization (adjust for 4 classes)
fig, ax = plt.subplots(2, 2, figsize=(10, 6), subplot_kw={"xticks": [], "yticks": []})
for i in range(0, 4):
    ax[i // 2, i % 2].imshow(x1_reshaped[i].reshape(4, 4), aspect="equal")
    predicted_label = np.argmax(y_predict[i])
    ax[i // 2, i % 2].set_title(f"The QCNN predicts this is {predicted_label}")
plt.subplots_adjust(wspace=0.1, hspace=0.5)
plt.show()

ModuleNotFoundError: No module named 'requests'